In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-07-01'
final_date = '2022-07-29'
query_error_year = 2022
query_error_month = 7

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\07. JULIO\DATA\ERROR JULIO 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 4)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 10 )
#             & (query_error['year'] == 2021), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(93811, 17)

In [5]:
query_error['mes_cierre'].unique()

array([nan, 'Actual'], dtype=object)

In [6]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [7]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [8]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [9]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_17148\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
82491,82491,eb06ecba-b875-4806-bdaf-d99622d59ee4,El Taco Keto,8LVBSGH3H,2022-07-01,277.89,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,eb06ecba-b875-4806-bdaf-d99622d59ee4,2022-07-01,1,Actual
82405,82405,ffde38ac-9444-41bb-a2d5-32d9fa54ab1a,Maxilofacial,8WHLJ9ZJV,2022-07-01,210.81,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,ffde38ac-9444-41bb-a2d5-32d9fa54ab1a,2022-07-01,1,Actual
82404,82404,f29390e2-1699-4bd6-96a5-de81f9368942,Lau,RHQB2D7MM,2022-07-01,0.95,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,f29390e2-1699-4bd6-96a5-de81f9368942,2022-07-01,1,Actual
82403,82403,98443ab6-ba8b-4ee5-bd57-eddfefdfbfe2,El Roble De Panuco,N4GZ8XVVP,2022-07-01,"1,505.41",ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,98443ab6-ba8b-4ee5-bd57-eddfefdfbfe2,2022-07-01,1,Actual
82402,82402,fad29aff-2483-4262-87a2-1db461b73c40,Make up yesica,4KJ3TPFTT,2022-07-01,0.95,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,fad29aff-2483-4262-87a2-1db461b73c40,2022-07-01,1,Actual


In [10]:
query_error_current_month.shape

(11703, 17)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# AGRUPADO POR ESQUEMA

In [14]:
group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only      amount                                 \
esquema_query                      ERROR_USUAL            SIN_CLABE             
status_cubeta                            ERROR        NEW     ERROR       NEW   
0                       2022-07-01        0.00  38,811.87      0.00 65,526.60   
1                       2022-07-02        0.00   4,234.44      0.00 76,627.47   
2                       2022-07-03   43,496.70 255,331.51      0.00 92,202.84   
3                       2022-07-04    3,399.63 331,805.43    205.98 63,139.92   
4                       2022-07-05        0.00  63,353.89      0.00 56,815.23   

                          
esquema_query        All  
status_cubeta             
0             104,338.47  
1              80,861.91  
2             391,031.05  
3             398,550.96  
4             120,169.12

In [15]:
group_esquema.shape

(30, 6)

In [16]:
writer = pd.ExcelWriter('FASE VI. DATA POLIZA ERROR-SIN CLABE JULIO 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
query_error_current_month.to_excel(writer, sheet_name='DETALLE',index=False)
writer.close()